In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Dikson,RU,2021-10-11 21:18:50,73.5069,80.5464,30.94,67,100,13.82
1,1,Port Maria,JM,2021-10-11 21:18:50,18.3685,-76.8895,85.96,79,32,6.20
2,2,Mataura,NZ,2021-10-11 21:13:51,-46.1927,168.8643,43.48,89,99,8.43
3,3,Punta Arenas,CL,2021-10-11 21:14:04,-53.1500,-70.9167,53.71,50,40,26.46
4,4,Peniche,PT,2021-10-11 21:18:51,39.3558,-9.3811,76.62,54,100,10.36


In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 75


In [12]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Dikson,RU,2021-10-11 21:18:50,73.5069,80.5464,30.94,67,100,13.82
2,2,Mataura,NZ,2021-10-11 21:13:51,-46.1927,168.8643,43.48,89,99,8.43
3,3,Punta Arenas,CL,2021-10-11 21:14:04,-53.1500,-70.9167,53.71,50,40,26.46
5,5,Coolum Beach,AU,2021-10-11 21:18:51,-26.5333,153.1000,67.10,98,100,6.87
6,6,Ushuaia,AR,2021-10-11 21:18:51,-54.8000,-68.3000,51.46,46,40,17.27
...,...,...,...,...,...,...,...,...,...,...
567,567,Weymouth,US,2021-10-11 21:22:15,42.2209,-70.9398,70.02,87,75,1.99
572,572,Sakaiminato,JP,2021-10-11 21:22:16,35.5500,133.2333,66.18,88,75,19.57
578,578,Roebourne,AU,2021-10-11 21:22:18,-20.7833,117.1333,71.78,47,0,6.29
580,580,Lasa,IT,2021-10-11 21:22:19,46.6166,10.7002,50.88,69,100,6.46


In [17]:
preferred_cities_df.dropna()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Dikson,RU,2021-10-11 21:18:50,73.5069,80.5464,30.94,67,100,13.82
2,2,Mataura,NZ,2021-10-11 21:13:51,-46.1927,168.8643,43.48,89,99,8.43
3,3,Punta Arenas,CL,2021-10-11 21:14:04,-53.1500,-70.9167,53.71,50,40,26.46
5,5,Coolum Beach,AU,2021-10-11 21:18:51,-26.5333,153.1000,67.10,98,100,6.87
6,6,Ushuaia,AR,2021-10-11 21:18:51,-54.8000,-68.3000,51.46,46,40,17.27
...,...,...,...,...,...,...,...,...,...,...
567,567,Weymouth,US,2021-10-11 21:22:15,42.2209,-70.9398,70.02,87,75,1.99
572,572,Sakaiminato,JP,2021-10-11 21:22:16,35.5500,133.2333,66.18,88,75,19.57
578,578,Roebourne,AU,2021-10-11 21:22:18,-20.7833,117.1333,71.78,47,0,6.29
580,580,Lasa,IT,2021-10-11 21:22:19,46.6166,10.7002,50.88,69,100,6.46


In [16]:
preferred_cities_df.count()

City_ID       324
City          324
Country       322
Date          324
Lat           324
Lng           324
Max Temp      324
Humidity      324
Cloudiness    324
Wind Speed    324
dtype: int64

In [18]:
#Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Dikson,RU,30.94,73.5069,80.5464,
2,Mataura,NZ,43.48,-46.1927,168.8643,
3,Punta Arenas,CL,53.71,-53.1500,-70.9167,
5,Coolum Beach,AU,67.10,-26.5333,153.1000,
6,Ushuaia,AR,51.46,-54.8000,-68.3000,
8,Kodiak,US,44.11,57.7900,-152.4072,
9,Nikolskoye,RU,42.87,59.7035,30.7861,
10,Banepa,NP,66.52,27.6298,85.5214,
12,Puerto Madryn,AR,62.38,-42.7692,-65.0385,
13,Tuktoyaktuk,CA,30.20,69.4541,-133.0374,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [20]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

Hotel not found... skipping.


In [21]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Dikson,RU,30.94,73.5069,80.5464,
2,Mataura,NZ,43.48,-46.1927,168.8643,
3,Punta Arenas,CL,53.71,-53.1500,-70.9167,
5,Coolum Beach,AU,67.10,-26.5333,153.1000,
6,Ushuaia,AR,51.46,-54.8000,-68.3000,
...,...,...,...,...,...,...
567,Weymouth,US,70.02,42.2209,-70.9398,
572,Sakaiminato,JP,66.18,35.5500,133.2333,
578,Roebourne,AU,71.78,-20.7833,117.1333,
580,Lasa,IT,50.88,46.6166,10.7002,


In [22]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [32]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))